<a href="https://colab.research.google.com/github/Paulina9555/Test/blob/master/pan_tadeusz.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import pandas as pd
import tensorflow.keras.backend as K

from tensorflow import keras
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Embedding, LSTM, Dense, Conv1D, MaxPool1D, Dropout, BatchNormalization, Bidirectional, Flatten
from tensorflow.keras.preprocessing import sequence
import tensorflow.keras.preprocessing.text as kpt


import pandas as pd
import numpy as np

In [ ]:
with open("pan_taduesz.txt","r", encoding = "UTF-8") as file:
    df = file.readlines()

In [ ]:
doc = [doc.replace("\n","").strip() for doc in df if len(doc) >= 30 and len(doc) <= 60 ]

In [ ]:
text = "".join(doc)

In [ ]:
# Wyodrębniamy sekwencje składajace się z 60 znaków.
maxlen = 60

# Nowa sekwencja jest próbkowana co 3 znaki.
step = 3

# Zmienna, w której zapisywane będą wyodrębnione sekwencje.
sentences = []

# Zmienna, w której zapisywane będą kolejne znaki (cele).
next_chars = []

for i in range(0, len(text) - maxlen, step):
    sentences.append(text[i: i + maxlen])
    next_chars.append(text[i + maxlen])
print('Liczba sekwencji:', len(sentences))

Liczba sekwencji: 141355


In [ ]:
len(sentences)

141355

In [ ]:
# Lista unikatowych znaków wchodzących w skład korpusu.
chars = sorted(list(set(text)))
print('Liczba unikatowych znaków:', len(chars))
# Słownik przypisujące unikatowe znaki do ich indeksów.
char_indices = dict((char, chars.index(char)) for char in chars)

Liczba unikatowych znaków: 83


In [ ]:
# Znaki są zapisywane w formie tablic binarnych przy użyciu kodowania z gorącą jedynką.
print('Tworzenie wektorów...')
x = np.zeros((len(sentences), maxlen, len(chars)), dtype=np.bool)
y = np.zeros((len(sentences), len(chars)), dtype=np.bool)
for i, sentence in enumerate(sentences):
    for t, char in enumerate(sentence):
        x[i, t, char_indices[char]] = 1
    y[i, char_indices[next_chars[i]]] = 1

Tworzenie wektorów...


<ipython-input-18-9741607de9ed>:3: DeprecationWarning: `np.bool` is a deprecated alias for the builtin `bool`. To silence this warning, use `bool` by itself. Doing this will not modify any behavior and is safe. If you specifically wanted the numpy scalar type, use `np.bool_` here.
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations
  x = np.zeros((len(sentences), maxlen, len(chars)), dtype=np.bool)
<ipython-input-18-9741607de9ed>:4: DeprecationWarning: `np.bool` is a deprecated alias for the builtin `bool`. To silence this warning, use `bool` by itself. Doing this will not modify any behavior and is safe. If you specifically wanted the numpy scalar type, use `np.bool_` here.
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations
  y = np.zeros((len(sentences), len(chars)), dtype=np.bool)


In [ ]:
x[0][0]

array([False, False, False, False, False, False, False, False, False,
       False, False, False, False, False, False, False, False, False,
       False, False, False, False, False,  True, False, False, False,
       False, False, False, False, False, False, False, False, False,
       False, False, False, False, False, False, False, False, False,
       False, False, False, False, False, False, False, False, False,
       False, False, False, False, False, False, False, False, False,
       False, False, False, False, False, False, False, False, False,
       False, False, False, False, False, False, False, False, False,
       False, False])

In [ ]:
model = Sequential([
    LSTM(32, input_shape=(maxlen, len(chars)), return_sequences = True),
    LSTM(64, input_shape=(maxlen, len(chars)), return_sequences = True),
    LSTM(128),
    Dense(len(chars), activation='softmax')
])

In [ ]:
optimizer = keras.optimizers.RMSprop(lr=0.01)
model.compile(loss='categorical_crossentropy', optimizer=optimizer, run_eagerly=True)

In [ ]:
def sample(preds, temperature=1.0):
    preds = np.asarray(preds).astype('float64')
    preds = np.log(preds) / temperature
    exp_preds = np.exp(preds)
    preds = exp_preds / np.sum(exp_preds)
    probas = np.random.multinomial(1, preds, 1)
    return np.argmax(probas)

In [ ]:
import random
import sys

for epoch in range(1, 100):
    print('epoch', epoch)
    # Jedna iteracja trenowania modelu na dostępnych danych treningowych.
    model.fit(x, y,
              batch_size=512,
              epochs=1)

    if epoch in [10, 20, 30, 40, 50, 60, 70, 80, 90, 99]:
        for temperature in [0.2, 0.5, 1.0, 1.2]:
            # Losowanie tekstu początkowego.
            start_index = random.randint(0, len(text) - maxlen - 1)
            generated_text = text[start_index: start_index + maxlen]
            generated_text = "Litwo! Ojczyzno moja! ty jesteś jak zdrowie:Ile cię trzeba c"
            print('--- Generowanie przy użyciu tekstu początkowego: "' + generated_text + '"')
            generated_text_all = generated_text
            print('------ Wartość parametru temperature:', temperature)

          # Generowanie 400 znaków (proces rozpoczyna się od wylosowanego tekstu początkowego).
            for i in range(400):
                sampled = np.zeros((1, maxlen, len(chars)))
                for t, char in enumerate(generated_text):
                    sampled[0, t, char_indices[char]] = 1.

                preds = model.predict(sampled, verbose=0)[0]
                next_index = sample(preds, temperature)
                next_char = chars[next_index]

                generated_text += next_char
                generated_text_all += next_char
                generated_text = generated_text[1:]

            print(len(generated_text_all))
            print(f"Wygenerowany text: {generated_text_all}")

epoch 1


277/277 [==============================] - 22s 73ms/step - loss: 3.4073
epoch 2
277/277 [==============================] - 19s 70ms/step - loss: 3.3747
epoch 3
277/277 [==============================] - 18s 65ms/step - loss: 3.3521
epoch 4
277/277 [==============================] - 19s 70ms/step - loss: 3.2138
epoch 5
277/277 [==============================] - 19s 67ms/step - loss: 3.0779
epoch 6
277/277 [==============================] - 18s 65ms/step - loss: 2.9608
epoch 7
277/277 [==============================] - 19s 69ms/step - loss: 2.8252
epoch 8
277/277 [==============================] - 19s 69ms/step - loss: 2.6998
epoch 9
277/277 [==============================] - 19s 69ms/step - loss: 2.6103
epoch 10
277/277 [==============================] - 18s 64ms/step - loss: 2.5462
--- Generowanie przy użyciu tekstu początkowego: "Litwo! Ojczyzno moja! ty jesteś jak zdrowie:Ile cię trzeba c"
------ Wartość parametru temperature: 0.2
460
Wygenerowany text: Litwo! Ojczyzno moja! ty jeste